# The Wold, Sheep and Cabbage Problem - Solved By BFS
Luciana Hoyos Pérez, Juan José Gómez Velez and Santiago Manco Maya

### Import collections

In [37]:
from collections import deque # Import deque for BFS implementation

### Node Definition

In [38]:
class Node: # Node class definition
    def __init__(self, state, parent=None, action=None):
        self.state = state # Current state of the node
        self.parent = parent # Parent node from which this node was created
        self.action = action # Action taken to reach this node

### Expand Function

In [39]:
def expand(problem, node):
    s = node.state # current state of the node
    # Generate successors of the current node
    for action in problem.actions: # Get all possible actions from the current state
        s_prime = problem.result(s, action) # Resulting state after applying the action
        if s_prime is not None: # Check if the resulting state is valid
            yield Node(state=s_prime, parent=node, action=action) # This creates a new node with the resulting state, the current node as parent, and the action taken to reach this state
            # The yield statement allows this function to be a generator, producing one node at a time

### BFS Function

In [40]:
def breadth_first_search(problem):
    node = Node(state=problem.initial) # Create the initial node with the problem's initial state
    frontier = deque([node])  # Initialize the frontier with the initial node (using deque for efficient FIFO operations)
    reached = {node.state} # Set to keep track of reached states to avoid duplicates
    # If the initial state is the goal, return the initial node
    if problem.is_goal(node.state):   # Check if the initial state is already a goal state
      return node

    while frontier:
        node = frontier.popleft() # Dequeue the first node from the frontier

        for child in expand(problem, node): # Generate successors of the current node
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached: # Check if the state has not been reached before
                reached.add(child.state)
                frontier.append(child) # Add the child node to the frontier for further exploration

    return None  # If no solution is found, return None

### Problem class

In [41]:
class Problem:
    def __init__(self, initial, goal, actions, result, is_goal):
        self.initial = initial # Initial state of the problem
        self.goal = goal # Goal state of the problem
        self.actions = actions # Function to get all possible actions from a state
        self.result = result  # Function to get the resulting state after applying an action
        self.is_goal = is_goal # Function to check if a state is the goal state

### Problem definition

In [42]:
def problem_definition():
    initial = ('L', 'L', 'L', 'L') # Initial state: all items (wolf, sheep, cabbage, farmer) on the left side
    goal = ('R', 'R', 'R', 'R') # Goal state: all items on the right side

    def moveFunction(state, index=None):
        new_state = list(state)

        if index is not None:
            # Move the specified item (wolf, sheep, or cabbage) with the farmer
            if new_state[3] != new_state[index]:
                return None  # Cannot move an item if it is not on the same side as the farmer
            new_side = 'R' if new_state[3] == 'L' else 'L'
            new_state[index] = new_side
            new_state[3] = new_side
        else:
            # Move the farmer alone
            new_side = 'R' if new_state[3] == 'L' else 'L'
            new_state[3] = new_side

        # Restrictions:
        # Wolf and sheep together without the farmer
        if new_state[0] == new_state[1] and new_state[3] != new_state[0]:
            return None
        # Sheep and cabbage together without the farmer
        if new_state[1] == new_state[2] and new_state[3] != new_state[1]:
            return None

        return tuple(new_state)

    
    actions = ["move_wolf", "move_sheep", "move_cabbage", "move_farmer"]

    def result(state, action):
        if action == "move_wolf":
            return moveFunction(state, 0)
        elif action == "move_sheep":
            return moveFunction(state, 1)
        elif action == "move_cabbage":
            return moveFunction(state, 2)
        elif action == "move_farmer":
            return moveFunction(state, None)

    
    def is_goal(state):
        return state == goal

    problem = Problem(initial, goal, actions, result, is_goal)

    solution = breadth_first_search(problem) # Perform the breadth-first search to find a solution

    def reconstruct_path(node):
        path = []
        while node.parent is not None:
            path.append((node.action, node.state))
            node = node.parent
        return list(reversed(path))

    if solution:
        path = reconstruct_path(solution)
        print(f"Initial state: {initial}")
        for action, state in path:
            print(f"Action: {action}, State: {state}") # Print each action and the resulting state
        print("The farmer successfully transported all items to the other side!")
    else:
        print("No solution found")

problem_definition()

Initial state: ('L', 'L', 'L', 'L')
Action: move_sheep, State: ('L', 'R', 'L', 'R')
Action: move_farmer, State: ('L', 'R', 'L', 'L')
Action: move_wolf, State: ('R', 'R', 'L', 'R')
Action: move_sheep, State: ('R', 'L', 'L', 'L')
Action: move_cabbage, State: ('R', 'L', 'R', 'R')
Action: move_farmer, State: ('R', 'L', 'R', 'L')
Action: move_sheep, State: ('R', 'R', 'R', 'R')
The farmer successfully transported all items to the other side!
